# This notebooks show how the networks are mapped based on a query

## 1. Import libraries and laod graph

In [12]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
from IPython.display import IFrame, display

# Load the graph you saved earlier
# (you can use the GEXF or GraphML file)
G = nx.read_gexf("materials_semantic_network.gexf")

print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Quick peek at some nodes
list(G.nodes(data=True))[:5]


Graph loaded: 6800 nodes, 36695 edges


[('optical tiles', {'category': 'source', 'label': 'optical tiles'}),
 ('raffia', {'category': 'source', 'label': 'raffia'}),
 ('pmma (acrylic)', {'category': 'source', 'label': 'pmma (acrylic)'}),
 ('beech wood', {'category': 'source', 'label': 'beech wood'}),
 ('recycled audio tape',
  {'category': 'source', 'label': 'recycled audio tape'})]

## 2. Build coocurrence graph

In [5]:
W1_TOL = 1e-6  # tolerance for weight ≈ 1.0

def build_cooccurrence_only_graph(G):
    H = nx.Graph()
    for n, d in G.nodes(data=True):
        H.add_node(n, **d)
    for u, v, d in G.edges(data=True):
        if d.get("edge_type") == "cooccurrence" and abs(d.get("weight",1.0) - 1.0) <= W1_TOL:
            H.add_edge(u, v, **d)
    return H

H = build_cooccurrence_only_graph(G)
print(f"Cooccurrence graph: {H.number_of_nodes()} nodes, {H.number_of_edges()} edges")


Cooccurrence graph: 6800 nodes, 10753 edges


## 3. Helpers to get categories and neighbors

In [6]:
def node_cat(G, n):
    return G.nodes[n].get("category")

def neighbors_by_cat(G, n, cat):
    return [m for m in G.neighbors(n) if node_cat(G, m) == cat]

def get_seed_role(G, seed):
    cat = node_cat(G, seed)
    if cat not in {"source", "application"}:
        raise ValueError("Seed must be a source or application node.")
    return cat


## 4. Function set for a seed

In [7]:
def function_set_for_seed(H, seed):
    """
    Given a seed (source OR application) in H, return:
      - its function neighbors (F_seed)
      - the opposite end nodes (applications if seed is source, or sources if seed is application)
      - the category of those end nodes
    """
    role = get_seed_role(H, seed)
    F_seed = neighbors_by_cat(H, seed, "function")

    if role == "source":
        end_nodes = set()
        for f in F_seed:
            end_nodes.update(neighbors_by_cat(H, f, "application"))
        end_cat = "application"
    else:
        end_nodes = set()
        for f in F_seed:
            end_nodes.update(neighbors_by_cat(H, f, "source"))
        end_cat = "source"

    return F_seed, sorted(end_nodes), end_cat


## 5. Function set similarity

In [8]:
import numpy as np

def avg_function_set_similarity(G, F_a, F_b):
    sims = []
    for fa in F_a:
        for fb in F_b:
            if fa == fb:
                sims.append(1.0)
            elif G.has_edge(fa, fb) and G[fa][fb].get("edge_type") == "sim_function":
                sims.append(float(G[fa][fb].get("weight", 0.0)))
    if not sims:
        return 0.0, 0
    sims = np.asarray(sims, dtype=np.float32)
    return float(sims.mean()), len(sims)


## 6. Build seed-centric subgraph

In [9]:
FUNC_SET_SIM_THRESHOLD = 0.9  # tune this as needed

def build_seed_thematic_subgraph(G, H, seed, threshold=FUNC_SET_SIM_THRESHOLD):
    if seed not in H:
        raise ValueError("Seed not found in the cooccurrence-only graph H.")

    role = get_seed_role(H, seed)
    F_seed, end_nodes_seed, end_cat = function_set_for_seed(H, seed)

    # Start subgraph
    S = nx.Graph()
    S.add_node(seed, **G.nodes[seed])

    # Add seed functions and end nodes
    for f in F_seed:
        S.add_node(f, **G.nodes[f])
        if H.has_edge(seed, f):
            S.add_edge(seed, f, **H[seed][f])
    for e in end_nodes_seed:
        S.add_node(e, **G.nodes[e])
        for f in F_seed:
            if H.has_edge(f, e):
                S.add_edge(f, e, **H[f][e])

    # Look for candidate end nodes of same type
    candidates = [n for n,d in H.nodes(data=True) if d.get("category") == end_cat]
    for cand in candidates:
        if cand in end_nodes_seed:
            continue
        F_cand = neighbors_by_cat(H, cand, "function")
        if not F_cand:
            continue
        avg_sim, n_pairs = avg_function_set_similarity(G, F_seed, F_cand)
        if n_pairs > 0 and avg_sim >= threshold:
            S.add_node(cand, **G.nodes[cand])
            for f in F_cand:
                S.add_node(f, **G.nodes[f])
                if H.has_edge(f, cand):
                    S.add_edge(f, cand, **H[f][cand])
                # also connect to opposite nodes
                if end_cat == "application":
                    opp_nodes = neighbors_by_cat(H, f, "source")
                else:
                    opp_nodes = neighbors_by_cat(H, f, "application")
                for opp in opp_nodes:
                    S.add_node(opp, **G.nodes[opp])
                    if H.has_edge(opp, f):
                        S.add_edge(opp, f, **H[opp][f])
    return S


## 7. Example run

In [10]:
# pick a real node ID from your graph that is a source or application
seed_node = "recycled denim"  # replace with one from your dataset

S = build_seed_thematic_subgraph(G, H, seed_node, threshold=0.35)
print(f"Subgraph from seed '{seed_node}': {S.number_of_nodes()} nodes, {S.number_of_edges()} edges")

# Inspect what was pulled in
list(S.nodes(data=True))[:10]


Subgraph from seed 'recycled denim': 12 nodes, 13 edges


[('recycled denim', {'category': 'source', 'label': 'recycled denim'}),
 ('combine-materials', {'category': 'function', 'label': 'combine-materials'}),
 ('offer-sustainability',
  {'category': 'function', 'label': 'offer-sustainability'}),
 ('provide-aesthetics',
  {'category': 'function', 'label': 'provide-aesthetics'}),
 ('architectural surface finish',
  {'category': 'application', 'label': 'architectural surface finish'}),
 ('automotive interior components',
  {'category': 'application', 'label': 'automotive interior components'}),
 ('coated textile for interior design',
  {'category': 'application', 'label': 'coated textile for interior design'}),
 ('eco-friendly interior wall coating',
  {'category': 'application', 'label': 'eco-friendly interior wall coating'}),
 ('interior design elements',
  {'category': 'application', 'label': 'interior design elements'}),
 ('outdoor architectural elements',
  {'category': 'application', 'label': 'outdoor architectural elements'})]

## 8. Visualization

In [14]:
HTML_OUT = "materials_cooccurrence_query.html"

net = Network(height="800px", width="100%", bgcolor="#ffffff", notebook=True, directed=False, cdn_resources='in_line')
net.toggle_physics(False)
# You can fine-tune physics via options if needed:
# net.set_options('{"physics":{"solver":"forceAtlas2Based","forceAtlas2Based":{"gravitationalConstant":-50}}}')

# Category colors
COLOR_BY_CATEGORY = {
    "source":      "#1f77b4",  # blue
    "function":    "#2ca02c",  # green
    "application": "#d62728",  # red
}

# Add nodes with category colors and degree-based size
for n, d in S.nodes(data=True):
    label = d.get("label", str(n))
    cat   = d.get("category", "unknown")
    color = COLOR_BY_CATEGORY.get(cat, "#888888")
    size  = 10 + 2 * S.degree(n)

    # Tooltip title shows category
    title = f"{cat.capitalize()}: {label}"
    net.add_node(n, label=label, color=color, title=title, size=size)

# Add edges (all are co-occurrence weight=1)
for u, v, d in S.edges(data=True):
    et = d.get("edge_type", "cooccurrence")
    net.add_edge(u, v, value=1, color="#555555", title=et)

net.show(HTML_OUT)
print("Graph written:", HTML_OUT)

from pathlib import Path

overlay_html = """
<div id="legend-box" style="
  position:fixed; right:20px; bottom:20px;
  background:#fff; border:1px solid #ccc; border-radius:6px;
  padding:10px 12px; font: 13px/1.2 sans-serif; z-index: 999999;
  box-shadow: 0 2px 8px rgba(0,0,0,0.15);">
  <div style="font-weight:600; margin-bottom:6px;">Legend</div>
  <div><span style="color:#4e79a7;">&#9679;</span> Source</div>
  <div><span style="color:#59a14f;">&#9679;</span> Function</div>
  <div><span style="color:#e15759;">&#9679;</span> Application</div>
</div>
"""

html_path = Path(HTML_OUT)
html = html_path.read_text(encoding="utf-8")

# Insert the overlay just before </body>
html = html.replace("</body>", overlay_html + "\n</body>")

html_path.write_text(html, encoding="utf-8")
print("Legend injected into:", HTML_OUT)

IFrame(src=HTML_OUT, width="100%", height="820")

materials_cooccurrence_query.html
Graph written: materials_cooccurrence_query.html
Legend injected into: materials_cooccurrence_query.html
